In [129]:
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt
from matplotlib import cm
import multiprocessing as mp

In [130]:
#Constructing a bump function for nvec_h and nvec_f
def bump(left_n, right_n, center, radius, S):
    left = left_n * np.ones(center - radius)
    right = right_n * np.ones(S - center - radius - 1)
    mean = (left_n + right_n) / 2
    middle = (right_n - mean) * np.sin(np.pi * (np.arange(center - radius, center + radius + 1) - center) / (2 * radius)) + mean
    n_vec = np.concatenate((left, middle, right))
    return n_vec

In [131]:
# Assume Cobb-Douglas. Generalized later. 

# Household parameters
life = 100
S = 100
working = round(9 * S / 16)
beta_annual = .96
beta = beta_annual ** (life/S)
sigma = 3.0

theta_h = 0.25
theta_f = 0.35

nvec_h = bump(1.0, 0.2, working, 5, S)
nvec_f = bump(1.0, 0.2, working, 5, S)

L_h = np.sum(nvec_h)
L_f = np.sum(nvec_f)

# Firm parameters
Z_h = 1.4
Z_f = 1.0

gamma_h = .35
gamma_f = .45

delta_annual_h = .05
delta_h = 1 - ((1 - delta_annual_h) ** (life/S))

delta_annual_f = .07
delta_f = 1 - ((1 - delta_annual_f) ** (life/S))

# SS parameters
SS_tol = 1e-13
xi = 0.9
EulDiff = True

In [132]:
def get_MUvec(cvec, sigma):
    epsilon = 0.0001
    b2 = (-sigma*(epsilon**(-sigma-1)))/2
    b1 = (epsilon**(-sigma))-2*b2*epsilon
    MU_vec = np.zeros_like(cvec)
    MU_vec[cvec<epsilon] = 2*b2*cvec[cvec<epsilon]+b1
    MU_vec[cvec>=epsilon] = cvec[cvec>=epsilon]**(-sigma)
    return MU_vec

#Get r, w_h, and w_f given aggregate_b

def get_r(K, L, Z, P, gamma, delta):
    return gamma*Z*P*((L/K)**(1 - gamma)) - delta

def get_w(K, L, Z, P, gamma):
    return (1 - gamma)*Z*P*((K/L)**gamma)

def get_P(theta, P_first, P_second, e, e_power):
    constant = ((1-theta)**(1-theta))*(theta**theta)
    return (1/constant)*(P_first**(1-theta))*(((e**e_power)*P_second)**theta)

def get_P_country(theta, P, agg_c, c):
    return P*(1-theta)*(agg_c/c)

In [133]:
#Get the SS values of b_vec_h and b_vec_f

def get_cvec(bvec, nvec, wvec, rvec, S, P, b_init=0):
    #
    #b_vec should be of size S - 1
    #n_vec should be of size S
    #w_vec should be of size S
    #r_vec should be of size S
    #
    #extended_b_vec should be of size S + 1, since it includes the 1st period
    #and the S + 1th period. Borrowing (or saving) in both these periods should be zero, by assumption.
    #
    extended_b_vec = np.zeros(S + 1)
    if b_init:
        extended_b_vec[0] = b_init
    extended_b_vec[1:-1] = bvec
    cvec = nvec * wvec + (1 + rvec) * extended_b_vec[:-1] - extended_b_vec[1:]
    return cvec/P

def solve_K(K, *args):
    P, Z, L, gamma, delta, r = args
    err_vec = r-get_r(K, L, Z, P, gamma, delta)
    return err_vec

def EulErr(bvec, *args):
    nvec, w, r, P, sigma, beta, EulDiff = args
    
    S = len(nvec)
    rvec = np.ones(S)*r
    wvec = np.ones(S)*w
    
    cvec = get_cvec(bvec, nvec, wvec, rvec, S, P)
    MU_vec = get_MUvec(cvec, sigma)
    
    if EulDiff:
        err_vec = MU_vec[:-1] - beta*(1+rvec[1:])*MU_vec[1:]
    else:
        err_vec = (beta*(1+rvec[1:])*MU_vec[1:]/MU_vec[:-1])-1
    return err_vec

def rerror(K_h, *args):
    aggregate_b, L_h, L_f, Z_h, Z_f, P_h, P_f, gamma_h, gamma_f, delta_h, delta_f = args
    r_h = get_r(K_h, L_h, Z_h, P_h, gamma_h, delta_h)
    K_f = aggregate_b - K_h
    r_f = get_r(K_f, L_f, Z_f, P_f, gamma_f, delta_f)
    return abs(r_h - r_f)

In [134]:
def get_SS(hh_params, firm_params, ss_params, guess_vec):
    beta, sigma, theta_h, theta_f, nvec_h, nvec_f, L_h, L_f = hh_params
    Z_h, Z_f, gamma_h, gamma_f, delta_h, delta_f = firm_params
    SS_tol, xi, EulDiff = ss_params
    old_vec = guess_vec
    err = 100
    iteration = 0
    while err > SS_tol:
        P_h = old_vec[0]
        P_f = old_vec[1]
        e = old_vec[2]
        r = old_vec[3]
        
        # Find wages using firm's problem
        K_h_args = (P_h, Z_h, L_h, gamma_h, delta_h, r)
        K_f_args = (P_f, Z_f, L_f, gamma_f, delta_f, r)
        K_guess = 0.8
        K_h = opt.fsolve(solve_K, K_guess, args=(K_h_args), xtol=SS_tol)
        K_f = opt.fsolve(solve_K, K_guess, args=(K_f_args), xtol=SS_tol)
        
        w_h = get_w(K_h, L_h, Z_h, P_h, gamma_h)
        w_f = get_w(K_f, L_f, Z_f, P_f, gamma_f)
        
        # Find implied aggregate prices 
        P = get_P(theta_h, P_h, P_f, e, -1)
        P_star = get_P(theta_f, P_f, P_h, e, 1)
        
        # Solve for savings using Euler Equations
        b_guess = np.ones(S-1)*0.1
        
        b_h_args = (nvec_h, w_h, r, P, sigma, beta, EulDiff)
        b_h = opt.fsolve(EulErr, b_guess, args=(b_h_args), xtol=SS_tol)
        
        b_f_args = (nvec_f, w_f, r, P_star, sigma, beta, EulDiff)
        b_f = opt.fsolve(EulErr, b_guess, args=(b_f_args), xtol=SS_tol)
        
        # Solve for consumption
        wvec_h = w_h*np.ones(S)
        wvec_f = w_f*np.ones(S)
        
        rvec = r*np.ones(S)
        
        c_h = get_cvec(b_h, nvec_h, wvec_h, rvec, S, P)
        c_f = get_cvec(b_f, nvec_f, wvec_f, rvec, S, P)
        
        c_hh = (1-theta_h)*((P_h/P)**(-1))*c_h
        c_hf = theta_h*((P_f/(P*e))**(-1))*c_h
        
        c_ff = (1-theta_f)*((P_f/P_star)**(-1))*c_f
        c_fh = theta_f*((e*P_h/P_star)**(-1))*c_f
        
        # Update K_h, K_f
        agg_b = np.sum(b_h+b_f)
        rerror_args = (agg_b, L_h, L_f, Z_h, Z_f, P_h, P_f, gamma_h, gamma_f, delta_h, delta_f)
        K_h = opt.fsolve(rerror, K_guess, args=(rerror_args), xtol=SS_tol)
        K_f = agg_b - K_h
        
        # Update r
        r_new = np.mean(get_r(K_h, L_h, Z_h, P_h, gamma_h, delta_h))
        
        # Update P_h, P_f
        # Prices are not updating for some reason!
        P_h_new = np.mean(get_P_country(theta_h, P, c_h, c_hh)) 
        P_f_new = np.mean(get_P_country(theta_f, P_star, c_f, c_ff))
        
        # Update exchange rate, e
        e_new = (P_f_new*np.sum(c_hf))/(P_h_new*np.sum(c_fh))
        
        # Calculate Euler Error
        iteration += 1
        
        new_vec = np.array([P_h_new, P_f_new, e_new, r_new])
        print(new_vec)
        
        err = np.sum(np.square(old_vec-new_vec))
        old_vec = xi*new_vec+(1-xi)*old_vec
        
        print("Iteration: ", iteration, "   Error: ", err)
    return old_vec, b_h, b_f

In [135]:
# Parameters
hh_params = (beta, sigma, theta_h, theta_f, nvec_h, nvec_f, L_h, L_f) 
firm_params = (Z_h, Z_f, gamma_h, gamma_f, delta_h, delta_f) 
ss_params = (SS_tol, xi, EulDiff)

# Initial guess for P_h, P_f, e, r
P_h_guess = 0.9
P_f_guess = 0.8
e_guess = 1.2
r_guess = 0.08

guess_vec = np.array([P_h_guess, P_f_guess, e_guess, r_guess])

guesses, bvec_h, bvec_f = get_SS(hh_params, firm_params, ss_params, guess_vec)

/Users/rubyzhang/anaconda/lib/python3.6/site-packages/scipy/optimize/minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


[  9.00000000e-01   8.00000000e-01   1.55820232e+00   1.53073745e-03]
Iteration:  1    Error:  0.13446632863
[ 0.9         0.8         1.91722673  0.06375924]
Iteration:  2    Error:  0.158859645822
[ 0.9         0.8         2.83175567  0.00980629]
Iteration:  3    Error:  0.912495268779
[ 0.9         0.8         4.5069484   0.03216418]
Iteration:  4    Error:  3.13530980608
[ 0.9         0.8         8.74964528  0.02264192]
Iteration:  5    Error:  19.5343060798
[  0.9          0.8         21.59673596   0.02613517]
Iteration:  6    Error:  176.599290787
[  9.00000000e-01   8.00000000e-01   7.31140088e+01   3.12263509e-02]
Iteration:  7    Error:  2792.71871271
[  9.00000000e-01   8.00000000e-01   4.12119004e+02   2.35227639e-02]
Iteration:  8    Error:  118535.337875
[  9.00000000e-01   8.00000000e-01   4.51366346e+03   2.12962199e-02]
Iteration:  9    Error:  17106276.124
[  9.00000000e-01   8.00000000e-01   1.27027656e+05   2.72690905e-02]
Iteration:  10    Error:  15111192407.3
[  9

/Users/rubyzhang/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:13: RuntimeWarning: invalid value encountered in power
/Users/rubyzhang/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:20: RuntimeWarning: invalid value encountered in double_scalars
/Users/rubyzhang/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:6: RuntimeWarning: invalid value encountered in less
/Users/rubyzhang/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:7: RuntimeWarning: invalid value encountered in greater_equal
